<a href="https://colab.research.google.com/github/salt24-web/03MAIR_Sergio_Salt/blob/main/Trabajo_seminario_Sergio_Salt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de otimización - Seminario

**Nombre y apellidos:** Sergio Salt Moya
<br>
**Github:** https://github.com/salt24-web/03MAIR_Sergio_Salt.git
<br>
**Google colab:** https://colab.research.google.com/drive/159g1ORsYyKIEtiM8GkKrWvQyskbzZCok?usp=sharing 



**Descripción del problema:**
<br>
Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice

¿Cuantas posiblidades hay sin tener en cuenta las restricciones? <br>
¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones? <br>
**Respuesta**
 <br>

-  Dado que hay 20 equipos y 10 partidos los cuales serán los que tenemos que repartir a lo largo de la jornada. El número de posibilidades son: **3628800** (10!)




**Modelo para el espacio de soluciones**
<br>
¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)
<br>
**Respuesta**
<br>
Ya que partimos con todos los datos y podemos repartirlos en varias matrices y solamente hay que generar un algortimo que los ordene de la forma más eficiente posible, la mejor estructura de datos pienso que es la que nos ofrece la librería **pandas** de python con los **DataFrames**. 

**Según el modelo para el espacio de soluciones**
<br>
¿Cual es la función objetivo?
<br>
f(x1,..., xn) = x1 · y1 + ... + xn · yn  
<br>
¿Es un problema de maximización o minimización?
<br>
Dado que se precisa encontrar la manera de obtener mejor audiencia, es un problema de maximización.

**Diseña un algoritmo para resolver el problema por fuerza bruta**

# Prepración de tablas.

In [1]:
import numpy as np
import pandas as pd
import sklearn

# Tabla horarios.

In [2]:

horarios = [
            ['Viernes' ,20 ,.4 ],
            ['Lunes'   ,20 ,.4 ],
            ['Sabado'  ,12 ,.55],
            ['Sabado'  ,16 ,.7 ],
            ['Sabado'  ,18 ,.8 ],
            ['Sabado'  ,20 , 1 ],
            ['Domingo' ,12 ,.45],
            ['Domingo' ,16 ,.75],
            ['Domingo' ,18 ,.5 ],
            ['Domingo' ,20 , 1 ]
]
df_horarios = pd.DataFrame(horarios, columns =["Dia", "Hora","Coeficiente"])
display(df_horarios)

,Dia,Hora,Coeficiente
0,Viernes,20,0.40
1,Lunes,20,0.40
2,Sabado,12,0.55
3,Sabado,16,0.70
4,Sabado,18,0.80
5,Sabado,20,1.00
6,Domingo,12,0.45
7,Domingo,16,0.75
8,Domingo,18,0.50
9,Domingo,20,1.00


# Tabla Equipos

In [3]:
equipos = [
            ['Real Madrid'   , 'A'],
            ['R. Sociedad'   , 'A'],
            [ 'Barcelona'    , 'A'],
            [ 'Villareal'    , 'B'],
            [  'Valencia'    , 'B'],
            [  'Alaves'      , 'B'],
            [  'Athletic'    , 'B'],
            [   'Celta'      , 'B'],
            [ 'Levante'      , 'B'],
            [ 'Espanyol'     , 'B'],
            [ 'Sevilla'      , 'B'],
            [  'Betis'       , 'B'],
            [ 'Atletico'     , 'B'],
            [  'Getafe'      , 'B'],
            [  'Mallorca'    , 'C'],
            [   'Eibar'      , 'C'],
            [  'Leganés'     , 'C'],
            [  'Osasuna'     , 'C'],
            [  'Granada'     , 'C'],
            ['Valladolid'    , 'C']          
]
df_equipos = pd.DataFrame(equipos, columns =["Equipo", "Categoria"])
display(df_equipos)

,Equipo,Categoria
0,Real Madrid,A
1,R. Sociedad,A
2,Barcelona,A
3,Villareal,B
4,Valencia,B
5,Alaves,B
6,Athletic,B
7,Celta,B
8,Levante,B
9,Espanyol,B


# Diccionario que almacena los niveles de audiencia según categoría.

In [4]:
dd_audiencia = [
    ['A-A',  2 ],
    ['A-B', 1.3],
    ['A-C',  1 ],
    ['B-B', .9 ],
    ['B-C',.75 ],
    ['C-C',.47 ]
]
df_dd_audiencia = pd.DataFrame(dd_audiencia, columns =["Codigo", "Audiencia"])
display(df_dd_audiencia)

,Codigo,Audiencia
0,A-A,2.00
1,A-B,1.30
2,A-C,1.00
3,B-B,0.90
4,B-C,0.75
5,C-C,0.47


# Diccionario que almacena el porcentaje de perdida según coincidencia.

In [5]:
dd_coincidencia = [
        [0, 0],
        [1, 25],
        [2, 45],
        [3, 60],
        [4, 70],
        [5, 75],
        [6, 78],
        [7, 80],
]
df_dd_coincidencia = pd.DataFrame(dd_coincidencia, columns =["N_Coincidencias", "Porcentaje_Perdida"])
display(df_dd_coincidencia)

,N_Coincidencias,Porcentaje_Perdida
0,0,0
1,1,25
2,2,45
3,3,60
4,4,70
5,5,75
6,6,78
7,7,80


# **Se crea la jornada aleatoria**

In [60]:

# Método para obtener la audiencia de la tabla
def get_audiencia_by_codigo(codigo):
    audiencia = df_dd_audiencia.loc[df_dd_audiencia['Codigo'] == codigo]
    if audiencia.empty:
      # En el caso de que no se encuentre el código significa que el codigo obtenido de forma aleatoria esta al revés.
      # Se invierte el orden para encontrar el código en el diccionario.
      audiencia = df_dd_audiencia.loc[df_dd_audiencia['Codigo'] == codigo[::-1]]

    return audiencia.Audiencia

# Método para obtener la jornada forma aleatoria.
def create_random_jornada(df_equipos):
  df_equipos_shuffle = sklearn.utils.shuffle(df_equipos)

  # Se inicializa el dataframe que contendrá la jornada.
  df_jornada = pd.DataFrame(data = [] , columns=('Equipos', 'Categoria', 'Audiencia'))
  for i in range(0, len(df_equipos_shuffle), 2):

    enfrentamiento = f"{df_equipos_shuffle.iloc[i].Equipo}-{df_equipos_shuffle.iloc[i + 1].Equipo}"
    categoria = f"{df_equipos_shuffle.iloc[i].Categoria}-{df_equipos_shuffle.iloc[i + 1].Categoria}"
    audiencia = get_audiencia_by_codigo(categoria)
    
    df_jornada = df_jornada.append(pd.DataFrame({
              'Equipos'    : enfrentamiento,
              'Categoria' : categoria,
              'Audiencia' : audiencia
    }), ignore_index = True)

  return df_jornada

df_jornada = create_random_jornada(df_equipos)

display(df_jornada)


,Equipos,Categoria,Audiencia
0,Atletico-Valladolid,B-C,0.75
1,Barcelona-Sevilla,A-B,1.30
2,Alaves-Leganés,B-C,0.75
3,Granada-Valencia,C-B,0.75
4,Eibar-Mallorca,C-C,0.47
5,Espanyol-Celta,B-B,0.90
6,Osasuna-Real Madrid,C-A,1.00
7,Getafe-Betis,B-B,0.90
8,Levante-Villareal,B-B,0.90
9,R. Sociedad-Athletic,A-B,1.30


# Algoritmo para ordenar una lista de menor a mayor.

In [18]:
def sort_list_upward_by_key(main_df, key):
  df_tmp = main_df.copy()
  df_sorted = main_df.drop(main_df.index)
  target = {}
  interacciones = 0

  while not(df_tmp.empty):
    interacciones += 1
    target.update({
        'index' : None,
        'target': df_tmp.iloc[0]
    })
    for i in range(len(df_tmp)):
      interacciones += 1
      row = df_tmp.iloc[i]
      if row[key] <= target['target'][key]:
        target.update({
            'index' : i,
            'target': row
        })

    df_sorted = df_sorted.append(target['target'], ignore_index = True)
    df_tmp = df_tmp.drop(target['index'])

    if not df_tmp.empty:
      df_tmp.reset_index(drop = True, inplace = True)

  print(f"Se han necesitado {interacciones} interacciones para ordenar la lista de menor a mayor.")
  return df_sorted


# Ordenación de partidos de menor a mayor según la audiendia.

In [20]:
df_jornada_sorted = sort_list_upward_by_key(df_jornada, 'Audiencia')
display(df_jornada_sorted)

Se han necesitado 65 interacciones para ordenar la lista de menor a mayor.


,Equipos,Categoria,Audiencia
0,Eibar-Leganés,C-C,0.47
1,Atletico-Valladolid,B-C,0.75
2,Mallorca-Valencia,C-B,0.75
3,Levante-Granada,B-C,0.75
4,Alaves-Athletic,B-B,0.90
5,Betis-Espanyol,B-B,0.90
6,Celta-Sevilla,B-B,0.90
7,R. Sociedad-Osasuna,A-C,1.00
8,Barcelona-Getafe,A-B,1.30
9,Real Madrid-Villareal,A-B,1.30


# Ordenación de horarios disponibles de menor a mayor según el coeficiente.

In [23]:
df_horarios_sorted = sort_list_upward_by_key(df_horarios, 'Coeficiente')
display(df_horarios_sorted)

Se han necesitado 65 interacciones para ordenar la lista de menor a mayor.


,Dia,Hora,Coeficiente
0,Lunes,20,0.40
1,Viernes,20,0.40
2,Domingo,12,0.45
3,Domingo,18,0.50
4,Sabado,12,0.55
5,Sabado,16,0.70
6,Domingo,16,0.75
7,Sabado,18,0.80
8,Domingo,20,1.00
9,Sabado,20,1.00


# Algoritmo para asignar a la jornada los horarios

In [30]:
def create_jornada_con_fecha(df_jornada, df_horarios):
  df_jornada_horarios = pd.DataFrame(data=[], columns = ['Enfrentamiento', 'Categoria', 'Audiencia_Inicial', 'Dia', 'Hora', 'Coeficiente', 'Audiencia_Final'])
  total = 0
  for i in range(0, len(df_jornada)) :
    df_h = df_horarios.iloc[i]
    df_j = df_jornada.iloc[i]
    audiencia_final = df_j.Audiencia * df_h.Coeficiente
    total += audiencia_final
    data = {
        'Enfrentamiento'      : df_j.Equipos,
        'Categoria'           : df_j.Categoria,
        'Audiencia_Inicial'   : df_j.Audiencia,
        'Dia'                 : df_h.Dia,
        'Hora'                : df_h.Hora,
        'Coeficiente'         : df_h.Coeficiente,
        'Audiencia_Final'     : audiencia_final
    }
    df_jornada_horarios = df_jornada_horarios.append(pd.DataFrame([data]))

  if not df_jornada_horarios.empty:
    df_jornada_horarios.reset_index(drop = True, inplace = True)


  return total, df_jornada_horarios



In [31]:
puntuacion, df_jornada_con_fecha = create_jornada_con_fecha(df_jornada_sorted, df_horarios_sorted)
display(df_jornada_con_fecha)
print(f"Puntuación total = {puntuacion}")

,Enfrentamiento,Categoria,Audiencia_Inicial,Dia,Hora,Coeficiente,Audiencia_Final
0,Eibar-Leganés,C-C,0.47,Lunes,20,0.40,0.1880
1,Atletico-Valladolid,B-C,0.75,Viernes,20,0.40,0.3000
2,Mallorca-Valencia,C-B,0.75,Domingo,12,0.45,0.3375
3,Levante-Granada,B-C,0.75,Domingo,18,0.50,0.3750
4,Alaves-Athletic,B-B,0.90,Sabado,12,0.55,0.4950
5,Betis-Espanyol,B-B,0.90,Sabado,16,0.70,0.6300
6,Celta-Sevilla,B-B,0.90,Domingo,16,0.75,0.6750
7,R. Sociedad-Osasuna,A-C,1.00,Sabado,18,0.80,0.8000
8,Barcelona-Getafe,A-B,1.30,Domingo,20,1.00,1.3000
9,Real Madrid-Villareal,A-B,1.30,Sabado,20,1.00,1.3000


Puntuación total = 6.4005


**Calcula la complejidad del algoritmo**
<br>
**Respuesta:** 
<br>
La complejidad es O(N*log N)

**Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta**

**Respuesta**

In [56]:
def calculate_audiencia(x):
  return x.Audiencia * x.Coeficiente


df_horarios_sorted = df_horarios.sort_values(by=['Coeficiente']).reset_index(drop = True)
df_jornada_sorted = df_jornada.sort_values(by=['Audiencia']).reset_index(drop = True)

df_jornada_con_fecha = pd.concat([df_jornada_sorted , df_horarios_sorted], axis =  1)
df_jornada_con_fecha.insert(loc = 6 ,value = None, column = "Audiencia_Final", allow_duplicates = True)
df_jornada_con_fecha['Audiencia_Final'] = df_jornada_con_fecha.apply(calculate_audiencia, axis=1)

display(df_jornada_con_fecha)
print(f"Puntuación total = {df_jornada_con_fecha.Audiencia_Final.sum()}")

,Equipos,Categoria,Audiencia,Dia,Hora,Coeficiente,Audiencia_Final
0,Eibar-Leganés,C-C,0.47,Viernes,20,0.40,0.1880
1,Levante-Granada,B-C,0.75,Lunes,20,0.40,0.3000
2,Mallorca-Valencia,C-B,0.75,Domingo,12,0.45,0.3375
3,Atletico-Valladolid,B-C,0.75,Domingo,18,0.50,0.3750
4,Celta-Sevilla,B-B,0.90,Sabado,12,0.55,0.4950
5,Betis-Espanyol,B-B,0.90,Sabado,16,0.70,0.6300
6,Alaves-Athletic,B-B,0.90,Domingo,16,0.75,0.6750
7,R. Sociedad-Osasuna,A-C,1.00,Sabado,18,0.80,0.8000
8,Real Madrid-Villareal,A-B,1.30,Sabado,20,1.00,1.3000
9,Barcelona-Getafe,A-B,1.30,Domingo,20,1.00,1.3000


Puntuación total = 6.4005


**Argumento**<br>

Con el uso de la librería pandas se puede realizar la misma operación con solamente 5 lineas. Además esta librería utiliza muchos módulos que están precompilados en el lenguaje de programación **C**, dada esta peculiaridad se crean operaciones mucho mas eficientes.

**Calcula la complejidad del algoritmo**

**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**
<br>
**Aplica el algoritmo al juego de datos generado**

Se utilizan las tablas y funciones anteriores para crear toda una liga con sus 38 jornadas, 20 equipos y su asignación de horarios.


In [63]:
max_total = {
    'jornada' : 0,
    'total' : 0
}
for i in range(1,39):
  df_jornada = create_random_jornada(df_equipos)

  df_horarios_sorted = df_horarios.sort_values(by=['Coeficiente']).reset_index(drop = True)
  df_jornada_sorted = df_jornada.sort_values(by=['Audiencia']).reset_index(drop = True)

  df_jornada_con_fecha = pd.concat([df_jornada_sorted , df_horarios_sorted], axis =  1)
  df_jornada_con_fecha.insert(loc = 6 ,value = None, column = "Audiencia_Final", allow_duplicates = True)

  df_jornada_con_fecha['Audiencia_Final'] = df_jornada_con_fecha.apply(calculate_audiencia, axis=1)
  df_jornada_con_fecha.style.set_caption(f'Jornada {i}')

  display(df_jornada_con_fecha)
  total = df_jornada_con_fecha.Audiencia_Final.sum()
  if max_total['total'] < total:
    max_total.update({'jornada' : i , 'total' : total})

  print(f"La puntuación de audiendia para la jornada {i} es de {total}")

print(f'La puntuación mas alta de audiencia se encuentra en la jornada {max_total["jornada"]} con una puntuación de {max_total["total"]}')

,Equipos,Categoria,Audiencia,Dia,Hora,Coeficiente,Audiencia_Final
0,Alaves-Granada,B-C,0.75,Viernes,20,0.40,0.3000
1,Valladolid-Betis,C-B,0.75,Lunes,20,0.40,0.3000
2,Mallorca-Villareal,C-B,0.75,Domingo,12,0.45,0.3375
3,Leganés-Atletico,C-B,0.75,Domingo,18,0.50,0.3750
4,Osasuna-Espanyol,C-B,0.75,Sabado,12,0.55,0.4125
5,Sevilla-Eibar,B-C,0.75,Sabado,16,0.70,0.5250
6,Getafe-Valencia,B-B,0.90,Domingo,16,0.75,0.6750
7,Athletic-Barcelona,B-A,1.30,Sabado,18,0.80,1.0400
8,Real Madrid-Levante,A-B,1.30,Sabado,20,1.00,1.3000
9,R. Sociedad-Celta,A-B,1.30,Domingo,20,1.00,1.3000


La puntuación de audiendia para la jornada 1 es de 6.5649999999999995


,Equipos,Categoria,Audiencia,Dia,Hora,Coeficiente,Audiencia_Final
0,Leganés-Celta,C-B,0.75,Viernes,20,0.40,0.3000
1,Villareal-Valladolid,B-C,0.75,Lunes,20,0.40,0.3000
2,Valencia-Osasuna,B-C,0.75,Domingo,12,0.45,0.3375
3,Granada-Sevilla,C-B,0.75,Domingo,18,0.50,0.3750
4,Getafe-Eibar,B-C,0.75,Sabado,12,0.55,0.4125
5,Levante-Espanyol,B-B,0.90,Sabado,16,0.70,0.6300
6,Atletico-Athletic,B-B,0.90,Domingo,16,0.75,0.6750
7,Barcelona-Mallorca,A-C,1.00,Sabado,18,0.80,0.8000
8,R. Sociedad-Alaves,A-B,1.30,Sabado,20,1.00,1.3000
9,Betis-Real Madrid,B-A,1.30,Domingo,20,1.00,1.3000


La puntuación de audiendia para la jornada 2 es de 6.43


,Equipos,Categoria,Audiencia,Dia,Hora,Coeficiente,Audiencia_Final
0,Osasuna-Valladolid,C-C,0.47,Viernes,20,0.40,0.1880
1,Espanyol-Leganés,B-C,0.75,Lunes,20,0.40,0.3000
2,Eibar-Valencia,C-B,0.75,Domingo,12,0.45,0.3375
3,Granada-Celta,C-B,0.75,Domingo,18,0.50,0.3750
4,Alaves-Mallorca,B-C,0.75,Sabado,12,0.55,0.4125
5,Sevilla-Betis,B-B,0.90,Sabado,16,0.70,0.6300
6,Levante-Athletic,B-B,0.90,Domingo,16,0.75,0.6750
7,Barcelona-Getafe,A-B,1.30,Sabado,18,0.80,1.0400
8,Villareal-R. Sociedad,B-A,1.30,Sabado,20,1.00,1.3000
9,Atletico-Real Madrid,B-A,1.30,Domingo,20,1.00,1.3000


...

La puntuación de audiendia para la jornada 36 es de 6.558


,Equipos,Categoria,Audiencia,Dia,Hora,Coeficiente,Audiencia_Final
0,Granada-Osasuna,C-C,0.47,Viernes,20,0.40,0.188
1,Levante-Mallorca,B-C,0.75,Lunes,20,0.40,0.300
2,Villareal-Atletico,B-B,0.90,Domingo,12,0.45,0.405
3,Alaves-Valencia,B-B,0.90,Domingo,18,0.50,0.450
4,Getafe-Espanyol,B-B,0.90,Sabado,12,0.55,0.495
5,Athletic-Betis,B-B,0.90,Sabado,16,0.70,0.630
6,Celta-Sevilla,B-B,0.90,Domingo,16,0.75,0.675
7,Barcelona-Leganés,A-C,1.00,Sabado,18,0.80,0.800
8,Real Madrid-Valladolid,A-C,1.00,Sabado,20,1.00,1.000
9,Eibar-R. Sociedad,C-A,1.00,Domingo,20,1.00,1.000


La puntuación de audiendia para la jornada 37 es de 5.943


,Equipos,Categoria,Audiencia,Dia,Hora,Coeficiente,Audiencia_Final
0,Mallorca-Eibar,C-C,0.47,Viernes,20,0.40,0.1880
1,Alaves-Osasuna,B-C,0.75,Lunes,20,0.40,0.3000
2,Valladolid-Levante,C-B,0.75,Domingo,12,0.45,0.3375
3,Leganés-Villareal,C-B,0.75,Domingo,18,0.50,0.3750
4,Atletico-Betis,B-B,0.90,Sabado,12,0.55,0.4950
5,Valencia-Athletic,B-B,0.90,Sabado,16,0.70,0.6300
6,Espanyol-Sevilla,B-B,0.90,Domingo,16,0.75,0.6750
7,Real Madrid-Granada,A-C,1.00,Sabado,18,0.80,0.8000
8,Getafe-R. Sociedad,B-A,1.30,Sabado,20,1.00,1.3000
9,Barcelona-Celta,A-B,1.30,Domingo,20,1.00,1.3000


La puntuación de audiendia para la jornada 38 es de 6.4005
La puntuación mas alta de audiencia se encuentra en la jornada 4 con una puntuación de 6.9085
